# Exploring the data downloaded from USDA FoodData Central

See the download here: https://fdc.nal.usda.gov/download-datasets.html

Data available in `.data/`.

Data dictionary available in  `nutrify/data_exploration/data/FoodData_Central_foundation_food_csv_2021-04-28/Download & API Field Descriptions April 2021.pdf`





In [126]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Get Data

In [288]:
# Import databases
food = pd.read_csv("data/FoodData_Central_foundation_food_csv_2021-04-28/food.csv")
nutrient = pd.read_csv("data/FoodData_Central_Supporting_Data_csv_2021-04-28/nutrient.csv")
food_nutrient = pd.read_csv("data/FoodData_Central_foundation_food_csv_2021-04-28/food_nutrient.csv")

/Users/daniel/code/nutrify/env/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [289]:
food.head()

,fdc_id,data_type,description,food_category_id,publication_date
0,319874,sample_food,"HUMMUS, SABRA CLASSIC",16.0,2019-04-01
1,319875,market_acquisition,"HUMMUS, SABRA CLASSIC",16.0,2019-04-01
2,319876,market_acquisition,"HUMMUS, SABRA CLASSIC",16.0,2019-04-01
3,319877,sub_sample_food,Hummus,16.0,2019-04-01
4,319878,sub_sample_food,Hummus,16.0,2019-04-01


In [290]:
nutrient.head()

,id,name,unit_name,nutrient_nbr,rank
0,1001,Solids,G,201.0,200.0
1,1002,Nitrogen,G,202.0,500.0
2,1003,Protein,G,203.0,600.0
3,1004,Total lipid (fat),G,204.0,800.0
4,1005,"Carbohydrate, by difference",G,205.0,1110.0


In [291]:
food_nutrient.head()

,id,fdc_id,nutrient_id,amount,data_points,derivation_id,min,max,median,footnote,min_year_acqured
0,2201847,319877,1051,56.30,1.0,1.0,NaN,NaN,NaN,NaN,NaN
1,2201845,319877,1002,1.28,1.0,1.0,NaN,NaN,NaN,NaN,NaN
2,2201846,319877,1004,19.00,1.0,1.0,NaN,NaN,NaN,NaN,NaN
3,2201844,319877,1007,1.98,1.0,1.0,NaN,NaN,NaN,NaN,NaN
4,2201852,319878,1091,188.00,1.0,1.0,NaN,NaN,NaN,NaN,NaN


In [128]:
# How many unique?
unique_descriptions = df["description"].unique()
len(unique_descriptions)

11368

Beautiful, this gives us ~11368 foods to work with as a goal to model. But surely they can be split into less categories?

In [129]:
unique_descriptions[:10]

array(['HUMMUS, SABRA CLASSIC', 'Hummus', 'HUMMUS, OTHER',
       'Hummus - NFY12140O', 'Hummus - NFY12140P', 'Hummus - NFY12140Q',
       'Hummus - NFY12140R', 'Hummus - NFY12140S', 'Hummus - NFY12140F',
       'Hummus - NFY12140G'], dtype=object)

Where do these descriptions come from?

How can we reduce them down to like 10 unique foods and keep it simple...

In [130]:
unique_descriptions[-10:]

array(['MUSHROOMS, LIONS MANE', 'OIL, PEANUT', 'OIL, SUNFLOWER',
       'OIL, SAFFLOWER', 'OIL, OLIVE, EXTRA LIGHT',
       'SPINACH, REGULAR (MATURE)', 'SPINACH, BABY', 'TOMATOES, ROMA',
       "MUSHROOMS, LION'S MANE", nan], dtype=object)

In [131]:
# Find random indexes of food to explore
import random
random_number = random.randint(0, len(unique)-10)
unique_descriptions[random_number:random_number+10]

array(['Tuna, canned, BUMBLEBEE CHUNK LIGHT, IN WATER, drained solids (NY) - NFY090S87',
       'Tuna, canned, BUMBLEBEE CHUNK LIGHT, IN WATER, drained solids (NY) - NFY090S8C',
       'Minerals, Tuna, canned, in water, drained solids,  BUMBLEBEE CHUNK LIGHT, IN WATER  (IN,NY) - NFY090S0A',
       'Proximates, Tuna, canned, in water, drained solids,  BUMBLEBEE CHUNK LIGHT, IN WATER  (IN,NY) - NFY090S0B',
       'Tuna, canned, in water, drained solids,  BUMBLEBEE CHUNK LIGHT, IN WATER  (CO,CT) - CY091JN',
       'Tuna, canned, BUMBLEBEE CHUNK LIGHT, IN WATER, drained solids (CT) - NFY090SBJ',
       'Tuna, canned, BUMBLEBEE CHUNK LIGHT, IN WATER, drained solids (CT) - NFY090SBN',
       'Tuna, canned, BUMBLEBEE CHUNK LIGHT, IN WATER, drained solids (CO) - NFY090SX2',
       'Tuna, canned, BUMBLEBEE CHUNK LIGHT, IN WATER, drained solids (CO) - NFY090SWY',
       'Tuna, canned, BUMBLEBEE CHUNK LIGHT, IN WATER, drained solids (CT) - NFY090SBM'],
      dtype=object)

In [132]:
df.columns

Index(['fdc_id', 'data_type', 'description', 'food_category_id',
       'publication_date'],
      dtype='object')

### Food Categories

Let's dive into food categories. 

In [133]:
unique_categories = df["food_category_id"].unique()
len(unique_categories)

19

19 different food categories... I wonder what these are?

In [134]:
df["food_category_id"].value_counts()

1.0     6406
9.0     3982
11.0    3788
4.0     2924
16.0    2450
5.0     1503
14.0     918
15.0     913
7.0      795
10.0     613
20.0     588
6.0      568
18.0     488
25.0     474
13.0     454
2.0      386
12.0     267
19.0      54
Name: food_category_id, dtype: int64

In [135]:
# Get food categories
food_cats = pd.read_csv("data/FoodData_Central_Supporting_Data_csv_2021-04-28/food_category.csv")
food_cats

,id,code,description
0,1,100,Dairy and Egg Products
1,2,200,Spices and Herbs
2,3,300,Baby Foods
3,4,400,Fats and Oils
4,5,500,Poultry Products
5,6,600,"Soups, Sauces, and Gravies"
6,7,700,Sausages and Luncheon Meats
7,8,800,Breakfast Cereals
8,9,900,Fruits and Fruit Juices
9,10,1000,Pork Products


## 10 foods we want

To keep things simple, we will reduce the databases from FoodData Central to 10 different foods.

Why these foods?

Because we have images for those foods ready to go.

```
["chicken_curry", "chicken_wings", "fried_rice", "grilled_salmon", "humburger", "ice_cream", "pizza", "ramen", "steak", "sushi"]
```

In [136]:
ten_foods = ["chicken_curry", 
    "chicken_wings", 
    "fried_rice", 
    "grilled_salmon", 
    "humburger", 
    "ice_cream", 
    "pizza", 
    "ramen", 
    "steak", 
    "sushi"]
ten_foods

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'humburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

In [144]:
food.head()

,fdc_id,data_type,description,food_category_id,publication_date
0,319874,sample_food,"HUMMUS, SABRA CLASSIC",16.0,2019-04-01
1,319875,market_acquisition,"HUMMUS, SABRA CLASSIC",16.0,2019-04-01
2,319876,market_acquisition,"HUMMUS, SABRA CLASSIC",16.0,2019-04-01
3,319877,sub_sample_food,Hummus,16.0,2019-04-01
4,319878,sub_sample_food,Hummus,16.0,2019-04-01


In [287]:
len(food)

12

In [192]:
# Drop rows with nulls
food = food.dropna()

# Foundation food is the ground truth for a certain type of food, excludes some details about the food
# E.g. the data_type foundation_food for Chicken will the the original unique ID for chicken
foundation_food = food[food["data_type"] == "foundation_food"]
len(foundation_food)

195

In [211]:
list(food[food["data_type"] == "sample_food"]["description"].unique())

['HUMMUS, SABRA CLASSIC',
 'HUMMUS, OTHER',
 'HUMMUS, TRIBE CLASSIC',
 'MILK, 2% (WAVE 22E)',
 'Beef, Eye of Round roast/steak, lean only, raw, animal 5, select',
 'Beef,  Eye of Round roast/steak, select, raw, comp13, lean (36BRERS/37BRERRS)',
 'Beef, Eye of Round roast/steak, lean only, raw, animal 10, select',
 'Beef, Eye of Round roast/steak, select, raw, comp5, lean (36BRERS/37BRERRS)',
 'Beef, Eye of Round roast/steak, lean only, raw, animal 11, select',
 'Beef, Eye of Round roast/steak, lean only, raw, animal 16, select',
 'Beef, Eye of Round roast/steak, lean only, raw, animal 17, select',
 'Beef, Eye of Round roast/steak, lean only, raw, animal 18, select',
 'Beef, Eye of Round roast/steak, select, raw, comp6, lean (36BRERS/37BRERRS)',
 'Beef, Eye of Round roast/steak, lean only, raw, animal 22, select',
 'Beef, Eye of Round roast/steak, lean only, raw, animal 23, select',
 'Beef, Eye of Round roast/steak, lean only, raw, animal 28, select',
 'Beef, Eye of Round roast/steak, l

In [197]:
foundation_foods = foundation_food["description"]
foundation_foods[20:40]

4153               Peanut butter, smooth style, with salt
4329                             Cheese, parmesan, grated
4491    Cheese, pasteurized process, American, vitamin...
4580    Grapefruit juice, white, canned or bottled, un...
4723                                 Peaches, yellow, raw
4817    Seeds, sunflower seed kernels, dry roasted, wi...
4951      Sausage, Italian, pork, mild, cooked, pan-fried
5164                  Bread, white, commercially prepared
5285          Sausage, turkey, breakfast links, mild, raw
5428                                        Cheese, swiss
5489    Kale, frozen, cooked, boiled, drained, without...
5751    Carrots, frozen, unprepared (Includes foods fo...
5991                            Mustard, prepared, yellow
6198                                Figs, dried, uncooked
6339                                Kiwifruit, green, raw
6491                              Melons, cantaloupe, raw
6650                                      Nectarines, raw
6794    Orange

In [203]:
# Found a list of the foundation foods we're going to start with!
list(foundation_foods)

['Hummus, commercial',
 'Milk, reduced fat, fluid, 2% milkfat, with added vitamin A and vitamin D',
 'Tomatoes, grape, raw',
 'Salt, table, iodized',
 'Beans, snap, green, canned, regular pack, drained solids',
 'Broccoli, raw',
 'Milk, lowfat, fluid, 1% milkfat, with added vitamin A and vitamin D',
 'Milk, nonfat, fluid, with added vitamin A and vitamin D (fat free or skim)',
 'Milk, whole, 3.25% milkfat, with added vitamin D',
 'Frankfurter, beef, unheated',
 'Nuts, almonds, dry roasted, with salt added',
 'Cheese, ricotta, whole milk',
 'Kale, raw',
 'Egg, whole, raw, frozen, pasteurized',
 'Egg, white, raw, frozen, pasteurized',
 'Egg, white, dried',
 'Sauce, salsa, ready-to-serve',
 'Sausage, breakfast sausage, beef, pre-cooked, unprepared',
 'Onion rings, breaded, par fried, frozen, prepared, heated in oven',
 'Pickles, cucumber, dill or kosher dill',
 'Peanut butter, smooth style, with salt',
 'Cheese, parmesan, grated',
 'Cheese, pasteurized process, American, vitamin D fortifi

In [212]:
# food.loc[(food["description"].str.contains("chicken", case=False)) & (food["description"].str.contains("drumstick", case=False))][-10:]
# Find chicken in foundation food
for food in foundation_foods:
    if "chicken" in food.lower():
        print(food)

Chicken, broilers or fryers, drumstick, meat only, cooked, braised
Chicken, broiler or fryers, breast, skinless, boneless, meat only, cooked, braised


In [224]:
chicken_wing_id = int(foundation_food.loc[foundation_food["description"].str.contains("Chicken", case=False)].iloc[0]["fdc_id"])
chicken_wing_id

331897

In [226]:
food_nutrient[food_nutrient["fdc_id"] == chicken_wing_id]

,id,fdc_id,nutrient_id,amount,data_points,derivation_id,min,max,median,footnote,min_year_acqured
41650,2259068,331897,1303,0.003,5.0,1.0,0.002,0.004,0.003,NaN,2010.0
41651,2259065,331897,1280,0.008,5.0,1.0,0.008,0.009,0.008,NaN,2010.0
41652,2259076,331897,1404,0.045,5.0,1.0,0.035,0.059,0.042,NaN,2010.0
41653,2259059,331897,1261,0.002,5.0,1.0,0.001,0.003,0.002,NaN,2010.0
41654,2259106,331897,1109,0.170,1.0,1.0,NaN,NaN,0.170,NaN,2010.0
...,...,...,...,...,...,...,...,...,...,...,...
41740,2259066,331897,1300,0.006,5.0,1.0,0.005,0.007,0.006,NaN,2010.0
41741,2259121,331897,1271,0.088,5.0,1.0,0.083,0.094,0.087,NaN,2010.0
41742,2259112,331897,1167,5.050,5.0,1.0,4.890,5.240,5.050,NaN,2010.0
41743,2259074,331897,1329,0.021,NaN,4.0,NaN,NaN,NaN,NaN,NaN


## Get protein, carb, fat IDs

See this document for info on foundation foods and their nutrients - https://fdc.nal.usda.gov/docs/Foundation_Foods_Documentation_Apr2021.pdf

* Carbohydrate, by difference = total carbohydrates


In [235]:
nutrient[(nutrient["name"].str.contains("protein", case=False)) | \
         (nutrient["name"].str.contains("carbohydrate", case=False)) | \
         (nutrient["name"].str.contains("fat", case=False))]

,id,name,unit_name,nutrient_nbr,rank
2,1003,Protein,G,203.0,600.0
3,1004,Total lipid (fat),G,204.0,800.0
4,1005,"Carbohydrate, by difference",G,205.0,1110.0
48,1049,"Solids, non-fat",G,253.0,999999.0
49,1050,"Carbohydrate, by summation",G,205.2,1120.0
52,1053,Adjusted Protein,G,257.0,700.0
70,1072,"Carbohydrate, other",G,284.0,NaN
83,1085,Total fat (NLEA),G,298.0,900.0
254,1257,"Fatty acids, total trans",G,605.0,15400.0
255,1258,"Fatty acids, total saturated",G,606.0,9700.0


In [236]:
target_nutrients = nutrient[nutrient["name"].isin(["Protein", "Total lipid (fat)", "Carbohydrate, by difference"])]
target_nutrients

,id,name,unit_name,nutrient_nbr,rank
2,1003,Protein,G,203.0,600.0
3,1004,Total lipid (fat),G,204.0,800.0
4,1005,"Carbohydrate, by difference",G,205.0,1110.0


In [271]:
target_nutrient_dict = {1003: "protein",
    1004: "fat",
    1005: "carbohydrate"
}

## Get target food protein, fat, carbohydrates

We want to now index on the target foods and the target nutrients and retrieve their values for each food/nutrient.

E.g.

```python
{"food_1": {"protein": 100,
            "carbohydrate": 50,
            "fat": 20},
 "food_2": ...

...}
```

In [275]:
list(target_nutrient_dict.keys())

[1003, 1004, 1005]

In [273]:
food_nutrient

,id,fdc_id,nutrient_id,amount,data_points,derivation_id,min,max,median,footnote,min_year_acqured,nutrient_name
0,2201847,319877,1051,56.30,1.0,1.0,NaN,NaN,NaN,NaN,NaN,Water
1,2201845,319877,1002,1.28,1.0,1.0,NaN,NaN,NaN,NaN,NaN,Nitrogen
2,2201846,319877,1004,19.00,1.0,1.0,NaN,NaN,NaN,NaN,NaN,Total lipid (fat)
3,2201844,319877,1007,1.98,1.0,1.0,NaN,NaN,NaN,NaN,NaN,Ash
4,2201852,319878,1091,188.00,1.0,1.0,NaN,NaN,NaN,NaN,NaN,"Phosphorus, P"
...,...,...,...,...,...,...,...,...,...,...,...,...
105684,21115567,1757386,2050,15.95,1.0,1.0,NaN,NaN,NaN,NaN,NaN,Genistin
105685,21115568,1757387,2050,37.47,1.0,1.0,NaN,NaN,NaN,NaN,NaN,Genistin
105686,21115569,1757388,2050,17.52,1.0,1.0,NaN,NaN,NaN,NaN,NaN,Genistin
105687,21115570,1757389,2050,11.06,1.0,1.0,NaN,NaN,NaN,NaN,NaN,Genistin


In [276]:
food_nutrient[(food_nutrient["nutrient_id"].isin(list(target_nutrient_dict.keys())))]

,id,fdc_id,nutrient_id,amount,data_points,derivation_id,min,max,median,footnote,min_year_acqured,nutrient_name
2,2201846,319877,1004,19.00,1.0,1.0,NaN,NaN,NaN,NaN,NaN,Total lipid (fat)
16,2201859,319882,1004,18.70,1.0,1.0,NaN,NaN,NaN,NaN,NaN,Total lipid (fat)
28,2201873,319892,1004,16.60,1.0,1.0,NaN,NaN,NaN,NaN,NaN,Total lipid (fat)
43,2201886,319899,1004,19.10,1.0,1.0,NaN,NaN,NaN,NaN,NaN,Total lipid (fat)
97,2201942,319908,1004,18.20,1.0,1.0,NaN,NaN,NaN,NaN,NaN,Total lipid (fat)
...,...,...,...,...,...,...,...,...,...,...,...,...
105606,21114599,1757307,1004,0.15,1.0,1.0,NaN,NaN,NaN,NaN,NaN,Total lipid (fat)
105613,21114606,1757314,1004,0.28,1.0,1.0,NaN,NaN,NaN,NaN,NaN,Total lipid (fat)
105642,21114635,1757343,1004,0.10,1.0,1.0,NaN,NaN,NaN,NaN,NaN,Total lipid (fat)
105652,21114645,1757353,1004,0.18,1.0,1.0,NaN,NaN,NaN,NaN,NaN,Total lipid (fat)


In [270]:
food_nutrient.dtypes

id                    int64
fdc_id                int64
nutrient_id           int64
amount              float64
data_points         float64
derivation_id       float64
min                 float64
max                 float64
median              float64
footnote             object
min_year_acqured    float64
nutrient_name        object
dtype: object

In [262]:
food_nutrient["nutrient_name"] = food_nutrient["nutrient_id"].map(nutrient.set_index("id")["name"])
food_nutrient

,id,fdc_id,nutrient_id,amount,data_points,derivation_id,min,max,median,footnote,min_year_acqured,nutrient_name
0,2201847,319877,1051,56.30,1.0,1.0,NaN,NaN,NaN,NaN,NaN,Water
1,2201845,319877,1002,1.28,1.0,1.0,NaN,NaN,NaN,NaN,NaN,Nitrogen
2,2201846,319877,1004,19.00,1.0,1.0,NaN,NaN,NaN,NaN,NaN,Total lipid (fat)
3,2201844,319877,1007,1.98,1.0,1.0,NaN,NaN,NaN,NaN,NaN,Ash
4,2201852,319878,1091,188.00,1.0,1.0,NaN,NaN,NaN,NaN,NaN,"Phosphorus, P"
...,...,...,...,...,...,...,...,...,...,...,...,...
105684,21115567,1757386,2050,15.95,1.0,1.0,NaN,NaN,NaN,NaN,NaN,Genistin
105685,21115568,1757387,2050,37.47,1.0,1.0,NaN,NaN,NaN,NaN,NaN,Genistin
105686,21115569,1757388,2050,17.52,1.0,1.0,NaN,NaN,NaN,NaN,NaN,Genistin
105687,21115570,1757389,2050,11.06,1.0,1.0,NaN,NaN,NaN,NaN,NaN,Genistin


In [277]:
# Find nutrition for chicken_wing_id (protein, fat, carb)
food_nutrient[(food_nutrient["fdc_id"] == chicken_wing_id) & (food_nutrient["nutrient_id"].isin(list(target_nutrient_dict.keys())))]

,id,fdc_id,nutrient_id,amount,data_points,derivation_id,min,max,median,footnote,min_year_acqured,nutrient_name
41686,2259098,331897,1004,5.95,6.0,1.0,5.54,6.33,5.93,NaN,2010.0,Total lipid (fat)
41718,2259079,331897,1003,23.90,NaN,49.0,23.00,24.60,24.10,NaN,NaN,Protein
41729,2259099,331897,1005,0.00,NaN,49.0,NaN,NaN,NaN,NaN,NaN,"Carbohydrate, by difference"


In [ ]:
ten_foods_ids = {chicken_curry}

In [278]:
ten_foods

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'humburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

In [284]:
sorted(list(foundation_foods))

['Almond milk, unsweetened, plain, shelf stable',
 'Apples, fuji, with skin, raw',
 'Apples, fuji, with skin, raw',
 'Apples, gala, with skin, raw',
 'Apples, gala, with skin, raw',
 'Apples, granny smith, with skin, raw',
 'Apples, granny smith, with skin, raw',
 'Apples, honeycrisp, with skin, raw',
 'Apples, honeycrisp, with skin, raw',
 'Apples, red delicious, with skin, raw',
 'Apples, red delicious, with skin, raw',
 'Bananas, overripe, raw',
 'Bananas, overripe, raw',
 'Bananas, ripe and slightly ripe, raw',
 'Bananas, ripe and slightly ripe, raw',
 'Beans, Dry, Black (0% moisture)',
 'Beans, Dry, Black (0% moisture)',
 'Beans, Dry, Brown (0% moisture)',
 'Beans, Dry, Brown (0% moisture)',
 'Beans, Dry, Carioca (0% moisture)',
 'Beans, Dry, Carioca (0% moisture)',
 'Beans, Dry, Cranberry (0% moisture)',
 'Beans, Dry, Cranberry (0% moisture)',
 'Beans, Dry, Dark Red Kidney (0% moisture)',
 'Beans, Dry, Dark Red Kidney (0% moisture)',
 'Beans, Dry, Flor de Mayo (0% moisture)',
 'B

In [301]:
ten_whole_foods = ["chicken_wings",
    "apple",
    "banana",
    "beef", # steak, etc
    "carrots",
    "egg", # whole egg
    "strawberries",
    "blueberries",
    "mushrooms",
    "honey"
]

In [293]:
# Find blueberries
food.head()

,fdc_id,data_type,description,food_category_id,publication_date
0,319874,sample_food,"HUMMUS, SABRA CLASSIC",16.0,2019-04-01
1,319875,market_acquisition,"HUMMUS, SABRA CLASSIC",16.0,2019-04-01
2,319876,market_acquisition,"HUMMUS, SABRA CLASSIC",16.0,2019-04-01
3,319877,sub_sample_food,Hummus,16.0,2019-04-01
4,319878,sub_sample_food,Hummus,16.0,2019-04-01


In [299]:
# Looks like blueberries and other foods are in the survery dataset: 
food = food.dropna()
list(food[food["description"].str.contains("berry", case=False)]["description"])

['Yogurt, Greek, strawberry, non-fat, DANNON OIKOS (CO,CT) CY120ZK',
 'Yogurt, Greek, strawberry, non-fat, DANNON OIKOS (CO1) NFY120NUA',
 'Yogurt, Greek, strawberry, non-fat, DANNON OIKOS (CO1) NFY120NUB',
 'Yogurt, Greek, strawberry, non-fat, DANNON OIKOS (CT1) NFY120ODF',
 'Yogurt, Greek, strawberry, non-fat, DANNON OIKOS (CT1) NFY120ODE',
 'Minerals, Yogurt, Greek, strawberry, non-fat, DANNON OIKOS (CO,CT) - NFY120ONZ',
 'Proximates, Yogurt, Greek, strawberry, non-fat, DANNON OIKOS (CO,CT) - NFY120OO0',
 'Yogurt, Greek, strawberry, non-fat, DANNON OIKOS (AL,CA1) CY120ZI',
 'Yogurt, Greek, strawberry, non-fat, DANNON OIKOS (AL1) NFY120O7W',
 'Yogurt, Greek, strawberry, non-fat, DANNON OIKOS (CA1) NFY120O9H',
 'Yogurt, Greek, strawberry, non-fat, DANNON OIKOS (AL1) NFY120O7X',
 'Yogurt, Greek, strawberry, non-fat, DANNON OIKOS (CA1) NFY120O9I',
 'Minerals, Yogurt, Greek, strawberry, non-fat, DANNON OIKOS (AL,CA1) - NFY120ON8',
 'Proximates, Yogurt, Greek, strawberry, non-fat, DANNON 

In [300]:
food

,fdc_id,data_type,description,food_category_id,publication_date
0,319874,sample_food,"HUMMUS, SABRA CLASSIC",16.0,2019-04-01
1,319875,market_acquisition,"HUMMUS, SABRA CLASSIC",16.0,2019-04-01
2,319876,market_acquisition,"HUMMUS, SABRA CLASSIC",16.0,2019-04-01
3,319877,sub_sample_food,Hummus,16.0,2019-04-01
4,319878,sub_sample_food,Hummus,16.0,2019-04-01
...,...,...,...,...,...
27580,1757377,sub_sample_food,"MUSHROOMS, SHIITAKE",11.0,2021-04-23
27581,1757378,sub_sample_food,"MUSHROOMS, SHIITAKE",11.0,2021-04-23
27582,1757379,sub_sample_food,"MUSHROOMS, SHIITAKE",11.0,2021-04-23
27583,1757380,sub_sample_food,"MUSHROOMS, SHIITAKE",11.0,2021-04-23


In [302]:
ten_whole_foods

['chicken_wings',
 'apple',
 'banana',
 'beef',
 'carrots',
 'egg',
 'strawberries',
 'blueberries',
 'mushrooms',
 'honey']

## Get ten whole foods `food_id`

Everything except blueberries and honey are available in `foundation_food`. 

For blueberries and honey, we'll have to dig into the survery data: `data_exploration/data/FoodData_Central_survey_food_csv_2020-10-30`

In [318]:
# Get all food ids from foundation_food (honey and blueberries in another dataset)
eight_whole_foods = ['apple', # removed chicken wings... can come back later...
 'banana',
 'beef',
 'carrots',
 'chicken',
 'egg',
 'strawberries',
 'mushrooms']

In [319]:
# str.contains can search on regex - https://stackoverflow.com/a/17973255/7900723
pattern = "|".join([f"(?i){food}" for food in eight_whole_foods])
pattern

'(?i)apple|(?i)banana|(?i)beef|(?i)carrots|(?i)chicken|(?i)egg|(?i)strawberries|(?i)mushrooms'

In [320]:
foundation_food[foundation_food["description"].str.contains(pattern, case=False)].sort_values(by=["description"])

,fdc_id,data_type,description,food_category_id,publication_date
20540,1105897,foundation_food,"Apples, fuji, with skin, raw",9.0,2020-10-30
20544,1750340,foundation_food,"Apples, fuji, with skin, raw",9.0,2020-10-30
20545,1750341,foundation_food,"Apples, gala, with skin, raw",9.0,2020-10-30
20425,1105781,foundation_food,"Apples, gala, with skin, raw",9.0,2020-10-30
20308,1105664,foundation_food,"Apples, granny smith, with skin, raw",9.0,2020-10-30
20546,1750342,foundation_food,"Apples, granny smith, with skin, raw",9.0,2020-10-30
20191,1105547,foundation_food,"Apples, honeycrisp, with skin, raw",9.0,2020-10-30
20547,1750343,foundation_food,"Apples, honeycrisp, with skin, raw",9.0,2020-10-30
20074,1105430,foundation_food,"Apples, red delicious, with skin, raw",9.0,2020-10-30
20543,1750339,foundation_food,"Apples, red delicious, with skin, raw",9.0,2020-10-30


In [316]:
sorted(eight_whole_foods)

['apple', 'banana', 'beef', 'carrots', 'egg', 'mushrooms', 'strawberries']

In [317]:
chicken_wing_id

331897

In [321]:
# Map foods to food_id (these have been filtered from larger quantities to smaller quantities)
# For example, if there were 5 kinds of apple, only one was chosen
eight_whole_foods_id_map = {1750339: "apple", # red delicious
    1105314: "banana", # Bananas, ripe and slightly ripe, raw	
    746763: "beef", # t-bone steak 
    746764: "carrots", # frozen unprepared
    331897: "chicken_wings", # Chicken, broilers or fryers, drumstick, meat o...	
    329490: "egg", # Egg, whole, dried	
    1750347: "mushrooms", # Mushrooms, white button
    747448: "strawberries" # strawberries, raw
}

In [325]:
list(eight_whole_foods_id_map.keys())

[1750339, 1105314, 746763, 746764, 331897, 329490, 1750347, 747448]

In [380]:
# Find nutrition for eight whole foods
eight_whole_foods_df = food_nutrient[(food_nutrient["fdc_id"].isin(list(eight_whole_foods_id_map.keys()))) & (food_nutrient["nutrient_id"].isin(list(target_nutrient_dict.keys())))][["fdc_id", "nutrient_id", "amount"]]
eight_whole_foods_df

,fdc_id,nutrient_id,amount
34265,329490,1004,39.800000
34266,329490,1005,1.870000
34270,329490,1003,48.100000
41686,331897,1004,5.950000
41718,331897,1003,23.900000
41729,331897,1005,0.000000
71052,746763,1003,27.300000
71079,746763,1005,0.000000
71097,746763,1004,11.400000
71175,746764,1004,0.470000


In [381]:
# Pivot the table to how we want it
eight_whole_foods_df = eight_whole_foods_df.pivot_table("amount", "fdc_id", "nutrient_id")
eight_whole_foods_df

nutrient_id,1003,1004,1005
fdc_id,,,
329490,48.100000,39.8000,1.870000
331897,23.900000,5.9500,0.000000
746763,27.300000,11.4000,0.000000
746764,0.810000,0.4700,7.920000
747448,0.640000,0.2200,7.630000
1105314,0.740000,0.2900,23.000000
1750339,0.187500,0.2125,14.781700
1750347,2.890625,0.3708,4.079375


In [382]:
eight_whole_foods_df = eight_whole_foods_df.reset_index(drop=False).rename_axis(None, axis=1)
eight_whole_foods_df

,fdc_id,1003,1004,1005
0,329490,48.100000,39.8000,1.870000
1,331897,23.900000,5.9500,0.000000
2,746763,27.300000,11.4000,0.000000
3,746764,0.810000,0.4700,7.920000
4,747448,0.640000,0.2200,7.630000
5,1105314,0.740000,0.2900,23.000000
6,1750339,0.187500,0.2125,14.781700
7,1750347,2.890625,0.3708,4.079375


In [383]:
target_nutrient_dict

{1003: 'protein', 1004: 'fat', 1005: 'carbohydrate'}

In [384]:
# Rename columns
eight_whole_foods_df.rename(columns=target_nutrient_dict, inplace=True)
eight_whole_foods_df

,fdc_id,protein,fat,carbohydrate
0,329490,48.100000,39.8000,1.870000
1,331897,23.900000,5.9500,0.000000
2,746763,27.300000,11.4000,0.000000
3,746764,0.810000,0.4700,7.920000
4,747448,0.640000,0.2200,7.630000
5,1105314,0.740000,0.2900,23.000000
6,1750339,0.187500,0.2125,14.781700
7,1750347,2.890625,0.3708,4.079375


In [386]:
# Add food names
eight_whole_foods_df["food_name"] = eight_whole_foods_df["fdc_id"].map(eight_whole_foods_id_map)
eight_whole_foods_df

,fdc_id,protein,fat,carbohydrate,food_name
0,329490,48.100000,39.8000,1.870000,egg
1,331897,23.900000,5.9500,0.000000,chicken_wings
2,746763,27.300000,11.4000,0.000000,beef
3,746764,0.810000,0.4700,7.920000,carrots
4,747448,0.640000,0.2200,7.630000,strawberries
5,1105314,0.740000,0.2900,23.000000,banana
6,1750339,0.187500,0.2125,14.781700,apple
7,1750347,2.890625,0.3708,4.079375,mushrooms


# Next

* add blueberries and honey to the database above
* train a model on the 10 foods...
* integrate model somewhere to show off the nutrients in each food! 